Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
# Загрузка библиотек

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import urllib.request
from pathlib import Path

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
class Dataset:

    def path_to_files(self, path, link):
        Path('datasets').mkdir(parents=True, exist_ok=True)
        def get_file(file_name, url):
            if not os.path.exists(file_name):
                print(file_name, 'файл не найден, будет загружен из сети')
                _ = urllib.request.urlretrieve(url, file_name)
        urls = {
            'dataset': (path, link)
        }
        [get_file(*urls[k]) for k in urls]
        data = pd.read_csv(urls['dataset'][0])
        return data

In [3]:
# Присвоим переменной ds класс Dataset

ds = Dataset()

# Загрузим данные

df = ds.path_to_files('insurance.csv','https://code.s3.yandex.net/datasets/insurance.csv')

In [4]:
# Изучим датасет

display(df.head())
display(df.info())
display(df.describe())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
# Приведем возраст и зарплату к int 

df['Возраст'] = df['Возраст'].astype(int)
df['Зарплата'] = df['Зарплата'].astype(int)
df.dtypes

Пол                  int64
Возраст              int64
Зарплата             int64
Члены семьи          int64
Страховые выплаты    int64
dtype: object

In [6]:
# Посмотрим на количество дубликатов в таблице

df.duplicated().sum()

153

In [7]:
# Посмотрим на количество пустых значений

df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [8]:
# Избавимся от дубликатов

df= df.drop_duplicates()

Предобработка данных выполнена. Данные приведены к нужным типам, дубликаты удалены, пропущенные значения проверены. Можно приступать к анализу данных.

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Нет, не изменится.

**Обоснование:** 

Формула расчета весов для линейной регрессии:

$$
w = (X^T X)^{-1} X^T y
$$

Представим новую матрицу признаков $X_{1}$ как произведение старой $X$ на матрицу $P$ (обратимую):

$$
X_{1} = X \otimes P
$$

Подставим новое значение $X_{1}$ в формулу $w_{1}$:

$$
w_{1} = ((X P)^T X P)^{-1} (X P)^T y
$$

Раскроем первое произведение $ (XP)^T $:

$$
w_{1} = (P^T X^T X P)^{-1} P^T X^T y
$$

Перегруппируем множители в скобках, чтобы это выглядело как произведение трех множителей:

$$
w_{1} = (P^T (X^T X) P)^{-1} P^T X^T y
$$

Раскроем скобки $ (P^T (X^T X) P)^{-1} $:

$$
w_{1} = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Так как $P$ по условию обратимая, то произведение $(P^T)^{-1} P^T$ равно $E$ (единичная матрица):

$$
w_{1} = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$

Можно заметить, что справа получилась формула для $w$:

$$
w_{1} = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w
$$

Подставим новое значение весов  $𝑤_{1}$  в формулу для предсказаний линейной регрессии.
Предсказания модели расчитываются по формуле:

$$
a = Xw
$$

Подставим в эту формулу значения $X_{1}$ и $w_{1}$ для того, чтобы на их основе вычислить предсказания модели $a_{1}$:

$$
a_{1} = X_{1}w_{1} = XPP^{-1} w
$$

Так как $P$ по условию обратимая, то произведение $PP^{-1}$ снова равно $E$:

$$
a_{1} = XPP^{-1} w = X E w = X w = a
$$

Мы доказали, что предсказания $a_{1}$ для матрицы признаков, умноженных на обратимую матрицу $P$ равны предсказаниям $a$.<br>
Следовательно, отвечая на вопрос изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу - нет, не изменится. В данном случае, параметры линейной регрессии в исходной задаче $w$ и в преобразованной $w_{1}$ связаны следующим образом:

$$
w_{1} = P^{-1} w
$$

## 3. Алгоритм преобразования

**Алгоритм**

Чтобы защитить данные и не потерять качество модели машинного обучения, нужно умножить признаки на обратимую матрицу $P$. 
Для этого алгоритм преобразования данных должен содержать процесс создания подобной матрицы $P$ и ее умножения на матрицу признаков. 

**Обоснование**

In [9]:
# Так как по условию задачи эта матрица при умножении на матрицу признаков должна давать матрицу такого же размера
# как исходная, то P должна быть квадратной с размером, равным количеству признаков, в нашем случае их 4.

random_matrix = np.random.randint(100,size = (4, 4))
random_matrix

array([[84, 29, 18, 55],
       [66, 48, 24, 93],
       [39, 71, 40, 52],
       [24, 22, 62, 97]])

In [10]:
# Проверим матрицу на обратимость 

matrix_inverted = np.linalg.inv(random_matrix)
matrix_inverted

array([[ 0.0209576 , -0.01036122, -0.00114091, -0.0013376 ],
       [-0.01118455,  0.00849514,  0.01615123, -0.01046146],
       [ 0.01692889, -0.03433216,  0.01087758,  0.01748627],
       [-0.01346921,  0.02258114, -0.01033356,  0.00183614]])

In [11]:
# Обозначим признаки и целевой признак

features = df.drop('Страховые выплаты', axis = 1)
target = df['Страховые выплаты']

In [12]:
# Рассчитаем веса для исходных признаков по стандартной формуле w = (X^T X)^{-1} X^T y

w = np.linalg.inv(features.T.dot(features)).dot(features.T).dot(target)

# Рассчитаем вектор предсказаний по исходным признакам по формуле 𝑎=𝑋𝑤

a = features @ w

# Проведем преобразование признаков

features_p = features @ random_matrix

# Рассчитаем веса  𝑤1  для преобразованных признаков по стандартной формуле  𝑤=(𝑋𝑇𝑋)−1𝑋𝑇𝑦

w1 = np.linalg.inv(features_p.T.dot(features_p)).dot(features_p.T).dot(target)

# Рассчитаем вектор предсказаний по преобразованным признакам по формуле  𝑎=𝑋𝑤

a1 = features_p @ w1

# Получим вектор разниц между предсказаниями по исходным признакам  𝑎  и преобразованным  𝑎1

diff = a - a1

# Просуммируем все элементы вектора diff, чтобы убедиться, что расхождения минимальны

diff.sum()

-0.00033814455821401523

Исходя из полученных данных мы видим, что сумма расхождений совсем маленькая (ее можно значительно уменьшить, если масштабировать признаки). Это значит $𝑎$ равен $𝑎_{1}$. Мы доказали, что преобразование не приводит к разным предсказаниям модели и не ухудшает качество модели.

## 4. Проверка алгоритма

In [13]:
# Создадим модель линейной регрессии для исходных данных и посмотрим ее оценку R2_Score

model = LinearRegression(normalize = True)
model.fit(features, target)
predictions = model.predict(features)
mse = mean_squared_error(target, predictions)
print('MSE Score:', mse)
print('R2_Score на исходных данных: ',r2_score(target, predictions))

MSE Score: 0.1252726382276536
R2_Score на исходных данных:  0.4302010046633359


In [14]:
# Преобразуем признаки умножив на матрицу P и рассчитаем R2_Score

features_matrix = features @ random_matrix
model.fit(features_matrix, target)
predictions_matrix = model.predict(features_matrix)
mse = mean_squared_error(target, predictions)
print('MSE Score:', mse)
print('R2_Score для преобразованных  признаков: ',r2_score(target, predictions_matrix))

MSE Score: 0.1252726382276536
R2_Score для преобразованных  признаков:  0.4302010046633371



Исходя их полученных результатов, можно сказать, что преобразованные данные схожи с результатами исходных данных. 
То есть, мы защитили данные пользователей и при этом не ухудшили качество модели машинного обучения.